In [1]:
import pandas as pd
#This contain 200 data which is labelled initially for training
train_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data_init.csv",index_col=False)
#This contain 87 data which is labelled for testing
test_data=pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\test_data.csv",index_col=False)

In [2]:
#cleaning the text
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import nltk
from nltk.corpus import stopwords
my_stopwords = set(stopwords.words('english') + ['super', 'duper', 'place'])
from nltk.tokenize import word_tokenize
import string
def clean(tex):
  tex = tex.replace("&amp;amp;", '').replace("\'",'') 
  tex=tex.str.replace(r'\d+','')
  tex = tex.str.replace('@user', '')
  tex = tex.str.replace("[^a-zA-Z#]", " " )
  tex = tex.str.replace("#", " ")
  tex = tex.str.replace("-", " ")
  tex = tex.str.replace("$", " ")
  tex = tex.apply(lambda x: x.split())
  tex= tex.apply(lambda x: [stemmer.stem(i) for i in x])
  def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in my_stopwords]
  tex = tex.apply(process)
  return tex

In [3]:
#Active learning
import scipy.sparse as sp
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier
def add_data(x,y):
    x=sp.vstack((x,y))
    return x
def add_label(x,y):
    x=np.concatenate((x,y))
    return x
def fit_model(x,y):
    solvers = ['liblinear']
    penalty = ['l2']
    c_values = [ 0.358]
    param_grid = dict(solver=solvers,penalty=penalty,C=c_values)
    #model=LogisticRegression(class_weight='balanced', random_state=42)
    base_estimator = LogisticRegression()
    model = HalvingGridSearchCV(base_estimator, param_grid, n_jobs=-1, scoring='accuracy',error_score=0)
    model.fit(x,y)
    return model
from scipy.stats import entropy
import numpy as np
def multi_argmax(values, n_instances=110):
    max_idx = np.argpartition(-values, n_instances-1, axis=0)[:n_instances]
    return max_idx
def entropy_sampling( X,n_instances= 110):
    ent=np.transpose(entropy(np.transpose(X)))
    return multi_argmax(ent, n_instances=n_instances)

In [4]:
#cleaning train and test data
train_data_text=clean(train_data['text'])
test_data_text=clean(test_data['text'])

<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [6]:
#Building Initial model and checking the accuracy
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=train_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(train_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=train_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)


length of training data:  198
length of testing data:  87
train accuracy :  68.68686868686868
test accuracy:  67.81609195402298


In [7]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data3.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)

<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  312
length of testing data:  87
train accuracy :  89.42307692307693
test accuracy:  79.3103448275862


In [8]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data4.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  412
length of testing data:  87
train accuracy :  92.71844660194175
test accuracy:  82.75862068965517


In [9]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data5.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  513
length of testing data:  87
train accuracy :  81.0916179337232
test accuracy:  77.01149425287356


In [10]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data25.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  568
length of testing data:  87
train accuracy :  74.82394366197182
test accuracy:  74.71264367816092


In [11]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data261.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)


<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  613
length of testing data:  87
train accuracy :  75.69331158238172
test accuracy:  74.71264367816092


In [12]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data12.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)


<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  708
length of testing data:  87
train accuracy :  85.45197740112994
test accuracy:  81.60919540229885


In [13]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data113.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  807
length of testing data:  87
train accuracy :  93.80421313506815
test accuracy:  83.9080459770115


In [14]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data114.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  905
length of testing data:  87
train accuracy :  94.69613259668508
test accuracy:  86.20689655172413


In [15]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data115.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  1001
length of testing data:  87
train accuracy :  95.00499500499501
test accuracy:  86.20689655172413


In [16]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data116.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)


<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  1098
length of testing data:  87
train accuracy :  94.1712204007286
test accuracy:  87.35632183908046


In [17]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data117.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  1197
length of testing data:  87
train accuracy :  93.81787802840435
test accuracy:  88.50574712643679


In [18]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data118.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  1296
length of testing data:  87
train accuracy :  92.97839506172839
test accuracy:  88.50574712643679


In [19]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data119.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  1394
length of testing data:  87
train accuracy :  93.32855093256815
test accuracy:  87.35632183908046


In [20]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data120.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  1499
length of testing data:  87
train accuracy :  92.12808539026017
test accuracy:  87.35632183908046


In [21]:
pool_data= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data121.csv",index_col=False)
train_data_text=clean(pool_data['text'])
test_data_text=clean(test_data['text'])
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(train_data_text.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label=pool_data['label'].to_numpy()
model=fit_model(x_train_tfidf,label)
print("length of training data: ", len(pool_data))
print("length of testing data: ", len(test_data))
x_test_counts = count_vect.transform(test_data_text.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions_train = model.predict(x_train_tfidf)
y_test=pool_data['label']
print('train accuracy : ',accuracy_score(y_test,predictions_train)*100)
predictions_test = model.predict(x_test_tfidf)
y_test=test_data['label']
print('test accuracy: ',accuracy_score(y_test,predictions_test)*100)



<ipython-input-2-9f415afe60e3>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  tex=tex.str.replace(r'\d+','')
<ipython-input-2-9f415afe60e3>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tex = tex.str.replace("[^a-zA-Z#]", " " )
<ipython-input-2-9f415afe60e3>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  tex = tex.str.replace("$", " ")


length of training data:  1600
length of testing data:  87
train accuracy :  93.75
test accuracy:  86.20689655172413


In [ ]:
#This is used to clean the unlabelld pool of reviews
clean_text_pool=clean(rev['text'])

In [ ]:
#This will help to create a new csv file in which we will have labelled text and queried text (which need to be labelled)
x_counts = count_vect.transform(clean_text_pool.apply(lambda x: ' '.join(x)))
x_tfidf = transformer.transform(x_counts)
sample=model.predict_proba(x_tfidf)
rev_id=entropy_sampling(sample)
y=[]
train_ai=rev.loc[rev_id]
train_ai.to_csv(r"C:\Users\shaha\Desktop\IBS\yelp\train_data121.csv", mode='a', index=False, header=False)